In [1]:
import os
import pyspark.sql.functions as F
from pyspark.sql import Window
from pyspark.sql import SparkSession

In [2]:
os.environ['SPARK_HOME'] = r'C:\spark\spark-3.5.4-bin-hadoop3'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
os.environ['PYSPARK_PYTHON'] = 'python'

In [3]:
spark = (
    SparkSession.builder
    .appName('Airport Traffic')
    .master('local[20]')
    .config('spark.executor.memory', '12g')
    .config('spark.executor.cors', '4')
    .config("spark.dynamicAllocation.enabled", "true")
    .config("spark.dynamicAllocation.minExecutors", "2")
    .config("spark.dynamicAllocation.maxExecutors", "20")
    .config('spark.executors.memoryOverhead', '2g')
    .config("spark.driver.memory", "12g")
    .config("spark.driver.maxResultSize", "4g")
    .config('spark.sql.adaptive.enabled', 'true')
    .config('spark.sql.adaptive.coalescePartitions.enabled', 'true')
    .config('spark.sql.adaptive.advisoryPartitionSizeInBytes', '128mb')# 128 default
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    .config('spark.dynamicAllocation.executorIdleTimeout', '120')
    .config('spark.sql.autoBroadcastJoinThreshold', '512mb')
    .getOrCreate()
)

In [4]:
df_path = r"F:\Datasets\CSV datasets\airport_traffic\*"

In [5]:
df = (
    spark
    .read
    .format('csv')
    .option('header', 'true')
    .option('inferSchema', 'true')
    .load(df_path)
)

In [6]:
spark

In [7]:
df.rdd.getNumPartitions()

18

In [8]:
df.count()

983842

In [9]:
df.createOrReplaceTempView('airport_traffic')

In [10]:
spark.sql("""
    SELECT
        count(*)
    FROM
        airport_traffic
""").show()

+--------+
|count(1)|
+--------+
|  983842|
+--------+



In [11]:
df.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- MONTH_NUM: integer (nullable = true)
 |-- MONTH_MON: string (nullable = true)
 |-- FLT_DATE: string (nullable = true)
 |-- APT_ICAO: string (nullable = true)
 |-- APT_NAME: string (nullable = true)
 |-- STATE_NAME: string (nullable = true)
 |-- FLT_DEP_1: integer (nullable = true)
 |-- FLT_ARR_1: integer (nullable = true)
 |-- FLT_TOT_1: integer (nullable = true)
 |-- FLT_DEP_IFR_2: integer (nullable = true)
 |-- FLT_ARR_IFR_2: integer (nullable = true)
 |-- FLT_TOT_IFR_2: integer (nullable = true)



In [12]:
df.show(5)

+----+---------+---------+--------+--------+----------+----------+---------+---------+---------+-------------+-------------+-------------+
|YEAR|MONTH_NUM|MONTH_MON|FLT_DATE|APT_ICAO|  APT_NAME|STATE_NAME|FLT_DEP_1|FLT_ARR_1|FLT_TOT_1|FLT_DEP_IFR_2|FLT_ARR_IFR_2|FLT_TOT_IFR_2|
+----+---------+---------+--------+--------+----------+----------+---------+---------+---------+-------------+-------------+-------------+
|2016|        1|      JAN|01-01-16|    LATI|    Tirana|   Albania|       24|       27|       51|         NULL|         NULL|         NULL|
|2016|        1|      JAN|01-01-16|    UDYZ|   Yerevan|   Armenia|        8|       15|       23|         NULL|         NULL|         NULL|
|2016|        1|      JAN|01-01-16|    LOWG|      Graz|   Austria|        6|        7|       13|         NULL|         NULL|         NULL|
|2016|        1|      JAN|01-01-16|    LOWI| Innsbruck|   Austria|       26|       32|       58|         NULL|         NULL|         NULL|
|2016|        1|      JAN|0

In [13]:
window = Window.partitionBy('APT_NAME')

df.withColumn(
    "FLT_TOT_1_SUM",
    F.sum('FLT_TOT_1').over(window)
).show()

+----+---------+---------+--------+--------+--------+----------+---------+---------+---------+-------------+-------------+-------------+-------------+
|YEAR|MONTH_NUM|MONTH_MON|FLT_DATE|APT_ICAO|APT_NAME|STATE_NAME|FLT_DEP_1|FLT_ARR_1|FLT_TOT_1|FLT_DEP_IFR_2|FLT_ARR_IFR_2|FLT_TOT_IFR_2|FLT_TOT_1_SUM|
+----+---------+---------+--------+--------+--------+----------+---------+---------+---------+-------------+-------------+-------------+-------------+
|2016|        1|      JAN|01-01-16|    EBAW| Antwerp|   Belgium|        4|        3|        7|         NULL|         NULL|         NULL|       116053|
|2016|        1|      JAN|02-01-16|    EBAW| Antwerp|   Belgium|        9|       11|       20|         NULL|         NULL|         NULL|       116053|
|2016|        1|      JAN|03-01-16|    EBAW| Antwerp|   Belgium|       10|       17|       27|         NULL|         NULL|         NULL|       116053|
|2016|        1|      JAN|04-01-16|    EBAW| Antwerp|   Belgium|       18|       17|       35|

In [14]:
spark.sql("""
    SELECT
        *,
        SUM(FLT_TOT_1) OVER (PARTITION BY APT_NAME) AS FLT_TOT_1_SUM
    FROM
        airport_traffic
""").show()

+----+---------+---------+--------+--------+--------+----------+---------+---------+---------+-------------+-------------+-------------+-------------+
|YEAR|MONTH_NUM|MONTH_MON|FLT_DATE|APT_ICAO|APT_NAME|STATE_NAME|FLT_DEP_1|FLT_ARR_1|FLT_TOT_1|FLT_DEP_IFR_2|FLT_ARR_IFR_2|FLT_TOT_IFR_2|FLT_TOT_1_SUM|
+----+---------+---------+--------+--------+--------+----------+---------+---------+---------+-------------+-------------+-------------+-------------+
|2016|        1|      JAN|01-01-16|    EBAW| Antwerp|   Belgium|        4|        3|        7|         NULL|         NULL|         NULL|       116053|
|2016|        1|      JAN|02-01-16|    EBAW| Antwerp|   Belgium|        9|       11|       20|         NULL|         NULL|         NULL|       116053|
|2016|        1|      JAN|03-01-16|    EBAW| Antwerp|   Belgium|       10|       17|       27|         NULL|         NULL|         NULL|       116053|
|2016|        1|      JAN|04-01-16|    EBAW| Antwerp|   Belgium|       18|       17|       35|

In [15]:
spark.sql("""
    SELECT
        *,
        RANK() OVER (
        PARTITION BY YEAR, APT_ICAO
        ORDER BY FLT_DEP_1 DESC
        ) AS RANK
    FROM
        airport_traffic
    ORDER BY
        YEAR, FLT_DEP_1 DESC
""").show()

+----+---------+---------+--------+--------+--------------------+-----------+---------+---------+---------+-------------+-------------+-------------+----+
|YEAR|MONTH_NUM|MONTH_MON|FLT_DATE|APT_ICAO|            APT_NAME| STATE_NAME|FLT_DEP_1|FLT_ARR_1|FLT_TOT_1|FLT_DEP_IFR_2|FLT_ARR_IFR_2|FLT_TOT_IFR_2|RANK|
+----+---------+---------+--------+--------+--------------------+-----------+---------+---------+---------+-------------+-------------+-------------+----+
|2016|        7|      JUL|25-07-16|    EHAM|Amsterdam - Schiphol|Netherlands|      782|      770|     1552|          782|          771|         1553|   1|
|2016|        7|      JUL|22-07-16|    EHAM|Amsterdam - Schiphol|Netherlands|      781|      776|     1557|          781|          756|         1537|   2|
|2016|        7|      JUL|08-07-16|    EHAM|Amsterdam - Schiphol|Netherlands|      780|      779|     1559|          780|          768|         1548|   3|
|2016|        7|      JUL|15-07-16|    EHAM|Amsterdam - Schiphol|Nethe

In [16]:
window = Window.partitionBy('YEAR', 'APT_ICAO').orderBy(F.col('FLT_DEP_1').desc())

df.withColumn(
    'RANK', F.rank().over(window)
).orderBy(
    'YEAR', F.col('FLT_DEP_1').desc()
).show()

+----+---------+---------+--------+--------+--------------------+-----------+---------+---------+---------+-------------+-------------+-------------+----+
|YEAR|MONTH_NUM|MONTH_MON|FLT_DATE|APT_ICAO|            APT_NAME| STATE_NAME|FLT_DEP_1|FLT_ARR_1|FLT_TOT_1|FLT_DEP_IFR_2|FLT_ARR_IFR_2|FLT_TOT_IFR_2|RANK|
+----+---------+---------+--------+--------+--------------------+-----------+---------+---------+---------+-------------+-------------+-------------+----+
|2016|        7|      JUL|25-07-16|    EHAM|Amsterdam - Schiphol|Netherlands|      782|      770|     1552|          782|          771|         1553|   1|
|2016|        7|      JUL|22-07-16|    EHAM|Amsterdam - Schiphol|Netherlands|      781|      776|     1557|          781|          756|         1537|   2|
|2016|        7|      JUL|08-07-16|    EHAM|Amsterdam - Schiphol|Netherlands|      780|      779|     1559|          780|          768|         1548|   3|
|2016|        7|      JUL|15-07-16|    EHAM|Amsterdam - Schiphol|Nethe

In [17]:
window = Window.partitionBy('YEAR').orderBy(F.col('FLT_DEP_1').desc())

df.withColumn(
    'rank', F.rank().over(window)
).filter(
    F.col('rank') == 1
).drop(
    'rank'
).show()

+----+---------+---------+--------+--------+--------------------+-----------+---------+---------+---------+-------------+-------------+-------------+
|YEAR|MONTH_NUM|MONTH_MON|FLT_DATE|APT_ICAO|            APT_NAME| STATE_NAME|FLT_DEP_1|FLT_ARR_1|FLT_TOT_1|FLT_DEP_IFR_2|FLT_ARR_IFR_2|FLT_TOT_IFR_2|
+----+---------+---------+--------+--------+--------------------+-----------+---------+---------+---------+-------------+-------------+-------------+
|2016|        7|      JUL|25-07-16|    EHAM|Amsterdam - Schiphol|Netherlands|      782|      770|     1552|          782|          771|         1553|
|2017|        5|      MAY|24-05-17|    EHAM|Amsterdam - Schiphol|Netherlands|      803|      767|     1570|          803|          771|         1574|
|2017|        8|      AUG|07-08-17|    EHAM|Amsterdam - Schiphol|Netherlands|      803|      783|     1586|          801|          794|         1595|
|2018|        8|      AUG|30-08-18|    LFPG|Paris-Charles-de-...|     France|      797|      785|   

In [18]:
spark.sql("""
WITH ranked_airports AS (
    SELECT
        *,
        RANK() OVER(
            PARTITION BY YEAR
            ORDER BY FLT_DEP_1 DESC
        ) AS rank
    FROM
        airport_traffic
    )
    SELECT
        *
    FROM
        ranked_airports
    WHERE
        rank = 1
""").show()

+----+---------+---------+--------+--------+--------------------+-----------+---------+---------+---------+-------------+-------------+-------------+----+
|YEAR|MONTH_NUM|MONTH_MON|FLT_DATE|APT_ICAO|            APT_NAME| STATE_NAME|FLT_DEP_1|FLT_ARR_1|FLT_TOT_1|FLT_DEP_IFR_2|FLT_ARR_IFR_2|FLT_TOT_IFR_2|rank|
+----+---------+---------+--------+--------+--------------------+-----------+---------+---------+---------+-------------+-------------+-------------+----+
|2016|        7|      JUL|25-07-16|    EHAM|Amsterdam - Schiphol|Netherlands|      782|      770|     1552|          782|          771|         1553|   1|
|2017|        5|      MAY|24-05-17|    EHAM|Amsterdam - Schiphol|Netherlands|      803|      767|     1570|          803|          771|         1574|   1|
|2017|        8|      AUG|07-08-17|    EHAM|Amsterdam - Schiphol|Netherlands|      803|      783|     1586|          801|          794|         1595|   1|
|2018|        8|      AUG|30-08-18|    LFPG|Paris-Charles-de-...|     

In [19]:
window = Window.partitionBy('APT_ICAO').orderBy(F.col('FLT_DATE').asc())

df.withColumn(
    'Running Sum',
    F.sum('FLT_TOT_1').over(window)
).show()

+----+---------+---------+--------+--------+------------+----------+---------+---------+---------+-------------+-------------+-------------+-----------+
|YEAR|MONTH_NUM|MONTH_MON|FLT_DATE|APT_ICAO|    APT_NAME|STATE_NAME|FLT_DEP_1|FLT_ARR_1|FLT_TOT_1|FLT_DEP_IFR_2|FLT_ARR_IFR_2|FLT_TOT_IFR_2|Running Sum|
+----+---------+---------+--------+--------+------------+----------+---------+---------+---------+-------------+-------------+-------------+-----------+
|2016|        1|      JAN|01-01-16|    EDDK|Cologne-Bonn|   Germany|       92|       90|      182|         NULL|         NULL|         NULL|        182|
|2017|        1|      JAN|01-01-17|    EDDK|Cologne-Bonn|   Germany|       95|       97|      192|         NULL|         NULL|         NULL|        374|
|2018|        1|      JAN|01-01-18|    EDDK|Cologne-Bonn|   Germany|      108|      110|      218|          108|          107|          215|        592|
|2019|        1|      JAN|01-01-19|    EDDK|Cologne-Bonn|   Germany|       97|    

In [20]:
spark.sql("""
    SELECT
        *,
        sum(FLT_TOT_1) OVER (
            PARTITION BY APT_ICAO
            ORDER BY FLT_DATE
        )
    FROM
        airport_traffic
""").show()

+----+---------+---------+--------+--------+------------+----------+---------+---------+---------+-------------+-------------+-------------+-------------------------------------------------------------------------------------------------------------------------------+
|YEAR|MONTH_NUM|MONTH_MON|FLT_DATE|APT_ICAO|    APT_NAME|STATE_NAME|FLT_DEP_1|FLT_ARR_1|FLT_TOT_1|FLT_DEP_IFR_2|FLT_ARR_IFR_2|FLT_TOT_IFR_2|sum(FLT_TOT_1) OVER (PARTITION BY APT_ICAO ORDER BY FLT_DATE ASC NULLS FIRST RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)|
+----+---------+---------+--------+--------+------------+----------+---------+---------+---------+-------------+-------------+-------------+-------------------------------------------------------------------------------------------------------------------------------+
|2016|        1|      JAN|01-01-16|    EDDK|Cologne-Bonn|   Germany|       92|       90|      182|         NULL|         NULL|         NULL|                                                     

# Calculate a 3-day moving average of FLT_TOT_1 for each airport (APT_ICAO), ordered by FLT_DATE. Handle potential gaps in dates gracefully.

In [21]:
window = Window.partitionBy('APT_ICAO').orderBy('FLT_DATE').rowsBetween(-2, 0)

df.withColumn(
    'moving_avg_3day',
    F.avg('FLT_TOT_1').over(window)
).orderBy('APT_ICAO', 'FLT_DATE').show()

+----+---------+---------+--------+--------+--------+----------+---------+---------+---------+-------------+-------------+-------------+------------------+
|YEAR|MONTH_NUM|MONTH_MON|FLT_DATE|APT_ICAO|APT_NAME|STATE_NAME|FLT_DEP_1|FLT_ARR_1|FLT_TOT_1|FLT_DEP_IFR_2|FLT_ARR_IFR_2|FLT_TOT_IFR_2|   moving_avg_3day|
+----+---------+---------+--------+--------+--------+----------+---------+---------+---------+-------------+-------------+-------------+------------------+
|2024|        1|      JAN|01-01-24|    BIKF|Keflavik|   Iceland|       76|       62|      138|         NULL|         NULL|         NULL|             138.0|
|2024|        2|      FEB|01-02-24|    BIKF|Keflavik|   Iceland|       62|       72|      134|         NULL|         NULL|         NULL|             136.0|
|2024|        3|      MAR|01-03-24|    BIKF|Keflavik|   Iceland|       76|       73|      149|         NULL|         NULL|         NULL|140.33333333333334|
|2024|        4|      APR|01-04-24|    BIKF|Keflavik|   Iceland|

In [33]:
spark.sql("""
    SELECT
        APT_ICAO,
        FLT_DATE,
        FLT_TOT_1,
        avg(FLT_TOT_1) OVER (
            PARTITION BY APT_ICAO
            ORDER BY FLT_DATE
            ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
        ) AS moving_avg_3day
    FROM
        airport_traffic
    ORDER BY
        APT_ICAO, FLT_DATE
""").show()

+--------+--------+---------+------------------+
|APT_ICAO|FLT_DATE|FLT_TOT_1|   moving_avg_3day|
+--------+--------+---------+------------------+
|    BIKF|01-01-24|      138|             138.0|
|    BIKF|01-02-24|      134|             136.0|
|    BIKF|01-03-24|      149|140.33333333333334|
|    BIKF|01-04-24|      167|             150.0|
|    BIKF|01-05-24|      164|             160.0|
|    BIKF|01-06-24|      198|176.33333333333334|
|    BIKF|01-07-24|      236|199.33333333333334|
|    BIKF|01-08-24|      232|             222.0|
|    BIKF|01-09-24|      235|234.33333333333334|
|    BIKF|01-10-24|      185|217.33333333333334|
|    BIKF|01-11-24|      166|195.33333333333334|
|    BIKF|01-12-24|      153|             168.0|
|    BIKF|02-01-24|      172|163.66666666666666|
|    BIKF|02-02-24|      100|141.66666666666666|
|    BIKF|02-03-24|      149|140.33333333333334|
|    BIKF|02-04-24|      159|             136.0|
|    BIKF|02-05-24|      160|             156.0|
|    BIKF|02-06-24| 

In [35]:
window = Window.orderBy(F.lit(1))

df.withColumn(
    'row_number',
    F.row_number().over(window)
).show()

+----+---------+---------+--------+--------+-------------+--------------------+---------+---------+---------+-------------+-------------+-------------+----------+
|YEAR|MONTH_NUM|MONTH_MON|FLT_DATE|APT_ICAO|     APT_NAME|          STATE_NAME|FLT_DEP_1|FLT_ARR_1|FLT_TOT_1|FLT_DEP_IFR_2|FLT_ARR_IFR_2|FLT_TOT_IFR_2|row_number|
+----+---------+---------+--------+--------+-------------+--------------------+---------+---------+---------+-------------+-------------+-------------+----------+
|2016|        1|      JAN|01-01-16|    LATI|       Tirana|             Albania|       24|       27|       51|         NULL|         NULL|         NULL|         1|
|2016|        1|      JAN|01-01-16|    UDYZ|      Yerevan|             Armenia|        8|       15|       23|         NULL|         NULL|         NULL|         2|
|2016|        1|      JAN|01-01-16|    LOWG|         Graz|             Austria|        6|        7|       13|         NULL|         NULL|         NULL|         3|
|2016|        1|      

In [39]:
spark.sql("""
    SELECT
        *,
        ROW_NUMBER() OVER (ORDER BY 1) AS row_number
    FROM
        airport_traffic
""").show()

+----+---------+---------+--------+--------+-------------+--------------------+---------+---------+---------+-------------+-------------+-------------+----------+
|YEAR|MONTH_NUM|MONTH_MON|FLT_DATE|APT_ICAO|     APT_NAME|          STATE_NAME|FLT_DEP_1|FLT_ARR_1|FLT_TOT_1|FLT_DEP_IFR_2|FLT_ARR_IFR_2|FLT_TOT_IFR_2|row_number|
+----+---------+---------+--------+--------+-------------+--------------------+---------+---------+---------+-------------+-------------+-------------+----------+
|2016|        1|      JAN|01-01-16|    LATI|       Tirana|             Albania|       24|       27|       51|         NULL|         NULL|         NULL|         1|
|2016|        1|      JAN|01-01-16|    UDYZ|      Yerevan|             Armenia|        8|       15|       23|         NULL|         NULL|         NULL|         2|
|2016|        1|      JAN|01-01-16|    LOWG|         Graz|             Austria|        6|        7|       13|         NULL|         NULL|         NULL|         3|
|2016|        1|      

In [41]:
window = Window.partitionBy('APT_NAME').orderBy(F.col('FLT_TOT_1').desc())

df.withColumn(
    'total_flights_rank',
    F.rank().over(window)
).show()

+----+---------+---------+--------+--------+--------+----------+---------+---------+---------+-------------+-------------+-------------+------------------+
|YEAR|MONTH_NUM|MONTH_MON|FLT_DATE|APT_ICAO|APT_NAME|STATE_NAME|FLT_DEP_1|FLT_ARR_1|FLT_TOT_1|FLT_DEP_IFR_2|FLT_ARR_IFR_2|FLT_TOT_IFR_2|total_flights_rank|
+----+---------+---------+--------+--------+--------+----------+---------+---------+---------+-------------+-------------+-------------+------------------+
|2016|        3|      MAR|31-03-16|    EBAW| Antwerp|   Belgium|       48|       52|      100|         NULL|         NULL|         NULL|                 1|
|2016|        4|      APR|04-04-16|    EBAW| Antwerp|   Belgium|       49|       47|       96|         NULL|         NULL|         NULL|                 2|
|2018|        4|      APR|17-04-18|    EBAW| Antwerp|   Belgium|       50|       45|       95|         NULL|         NULL|         NULL|                 3|
|2016|        3|      MAR|24-03-16|    EBAW| Antwerp|   Belgium|

In [47]:
window = Window.orderBy(F.col('FLT_TOT_1').desc())

df \
.withColumn('total_flights_rank', F.rank().over(window)) \
.withColumn('total_flights_dense_rank', F.dense_rank().over(window)) \
.select('APT_NAME', 'FLT_TOT_1', 'total_flights_rank', 'total_flights_dense_rank') \
.orderBy('total_flights_rank') \
.show(20)

+--------------------+---------+------------------+------------------------+
|            APT_NAME|FLT_TOT_1|total_flights_rank|total_flights_dense_rank|
+--------------------+---------+------------------+------------------------+
|iGA Istanbul Airport|     1687|                 1|                       1|
|iGA Istanbul Airport|     1646|                 2|                       2|
|           Frankfurt|     1628|                 3|                       3|
|Paris-Charles-de-...|     1616|                 4|                       4|
|            Istanbul|     1612|                 5|                       5|
|           Frankfurt|     1610|                 6|                       6|
|Paris-Charles-de-...|     1607|                 7|                       7|
|           Frankfurt|     1602|                 8|                       8|
|    Madrid - Barajas|     1600|                 9|                       9|
|Amsterdam - Schiphol|     1599|                10|                      10|